## Import libraries

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
pd.set_option('display.max_columns', 100)


import tensorflow as tf 
print(tf.__version__)


from sklearn.metrics import mean_absolute_percentage_error as MAPE

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dropout, BatchNormalization
from tensorflow.keras.layers import Flatten, Concatenate
from tensorflow.keras.layers import Input, Dense, Conv1D, MaxPooling1D, UpSampling1D, BatchNormalization, LSTM

from tensorflow.keras.utils import plot_model
from keras.models import save_model, load_model
from joblib import dump, load


def mrpe(y_true, preds):
    relError = np.mean(
        abs(y_true[y_true > 1e-3] - preds[y_true > 1e-3]) / y_true[y_true > 1e-3]
    )
    return relError

2.6.0


In [ ]:
fts = ['.porosityConnected',
     '.area_by_valume',
     '.porosity_local.mean',
     '.porosity_local.std',
     '.porosity_local.mode',
     '.porosity_micp.mean',
     '.porosity_micp.std',
     '.porosity_micp.mode',
     '_qmicp0.25',
     '_qmicp0.5',
     '_qmicp0.75',
     '_qlocal0.25',
     '_qlocal0.5',
     '_qlocal0.75',
    ]

In [ ]:
data = pd.read_csv('data_final.csv')

In [ ]:
data.head()

,index,total_porosity,absolute_permeability,formation_factor,key,.porosityConnected,.area_by_valume,.porosity_local.mean,.porosity_local.std,.porosity_local.mode,.porosity_micp.mean,.porosity_micp.std,.porosity_micp.mode,_qmicp0.25,_qmicp0.5,_qmicp0.75,_qlocal0.25,_qlocal0.5,_qlocal0.75,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,...,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,Soi,index_x,a,b,krw_max,sor
0,/home/sirius/00_ikt/data_parsing/no_erosion/Ki...,0.216522,482.935,21.4433,Kirby13,0.211851,30093.052469,22.131000,10.880222,20.25,17.810768,9.657943,15.75,12.479358,15.710703,19.741443,15.204028,20.384081,26.945441,0.002267,0.002264,0.002261,0.002264,0.002243,0.002216,0.002202,0.002189,0.002206,0.002258,0.002334,0.002409,0.002486,0.002539,0.002555,0.002574,0.002581,0.002575,0.002529,0.002486,0.002457,0.002430,0.002433,0.002418,0.002407,0.002398,0.002385,0.002381,0.002359,0.002326,0.002276,...,0.002339,0.002338,0.002337,0.002326,0.002290,0.002250,0.002184,0.002179,0.002163,0.002167,0.002180,0.002220,0.002249,0.002278,0.002321,0.002354,0.002394,0.002415,0.002415,0.002393,0.002378,0.002377,0.002383,0.002376,0.002332,0.002290,0.002271,0.002274,0.002240,0.002201,0.002169,0.002134,0.002115,0.002111,0.002154,0.002190,0.002206,0.002180,0.002123,0.002060,0.001983,0.001907,0.001869,0.001840,0.321419,/home/sirius/00_ikt/data_parsing/no_erosion/Ki...,3.611006,1.259497,0.534967,0.239906
1,/home/sirius/00_ikt/data_parsing/no_erosion/Ki...,0.216079,527.475,18.7206,Kirby9,0.210722,30332.324074,21.465415,9.101433,20.25,16.478338,6.570775,15.75,12.416005,15.375915,19.448717,15.126773,20.426078,26.871444,0.002247,0.002244,0.002251,0.002252,0.002248,0.002215,0.002191,0.002187,0.002171,0.002140,0.002098,0.002064,0.002018,0.001986,0.001962,0.001945,0.001965,0.001990,0.002031,0.002084,0.002128,0.002155,0.002150,0.002132,0.002098,0.002054,0.002011,0.001956,0.001913,0.001878,0.001851,...,0.002153,0.002131,0.002101,0.002073,0.002070,0.002087,0.002105,0.002159,0.002196,0.002230,0.002271,0.002305,0.002302,0.002317,0.002312,0.002292,0.002283,0.002294,0.002298,0.002303,0.002301,0.002292,0.002271,0.002242,0.002206,0.002171,0.002145,0.002100,0.002063,0.002063,0.002067,0.002072,0.002080,0.002092,0.002109,0.002117,0.002115,0.002098,0.002107,0.002093,0.002088,0.002095,0.002088,0.002076,0.319978,/home/sirius/00_ikt/data_parsing/no_erosion/Ki...,2.903981,1.659004,0.412284,0.253856
2,/home/sirius/00_ikt/data_parsing/no_erosion/Ki...,0.221126,480.012,21.4524,Kirby21,0.210641,29575.425926,22.055133,9.821042,20.25,17.814278,7.627593,20.25,13.042620,16.955674,20.244408,15.282709,20.724134,27.646900,0.001754,0.001773,0.001776,0.001776,0.001806,0.001833,0.001876,0.001911,0.001938,0.001971,0.002024,0.002086,0.002116,0.002137,0.002155,0.002195,0.002255,0.002284,0.002333,0.002392,0.002447,0.002444,0.002391,0.002343,0.002324,0.002305,0.002290,0.002263,0.002238,0.002210,0.002192,...,0.001879,0.001952,0.002004,0.002050,0.002079,0.002097,0.002117,0.002132,0.002130,0.002124,0.002117,0.002087,0.002072,0.002045,0.002030,0.002013,0.002005,0.001985,0.001963,0.001938,0.001903,0.001872,0.001830,0.001811,0.001796,0.001787,0.001762,0.001735,0.001747,0.001770,0.001779,0.001792,0.001822,0.001842,0.001860,0.001863,0.001878,0.001888,0.001913,0.001935,0.001969,0.002016,0.002082,0.002153,0.377660,/home/sirius/00_ikt/data_parsing/no_erosion/Ki...,2.744394,1.529140,0.401164,0.273753
3,/home/sirius/00_ikt/data_parsing/no_erosion/Ki...,0.228149,581.860,18.5735,Kirby15,0.217476,30369.688272,22.325847,10.569668,20.25,16.782527,6.152925,15.75,12.763110,16.213480,19.730502,15.527208,20.843162,27.680412,0.001857,0.001899,0.001930,0.001963,0.001996,0.002008,0.001998,0.001993,0.001976,0.001935,0.001906,0.001884,0.001851,0.001822,0.001793,0.001784,0.001797,0.001834,0.001872,0.001914,0.001947,0.001987,0.002025,0.002070,0.002106,0.002

In [ ]:
# функция для пресказаний, используя нейронную сеть
def predict_with_nn(models, scalers_, X):
    preds = []
    for model, scalers in zip(models, scalers_):
        inp1 = scalers[0].transform(X[[str(x) for x in range(300)]])
        inp2 = scalers[1].transform(X[fts])
        preds.append(model.predict([inp1, inp2]).reshape(-1))
    return np.mean(preds, axis = 0)
# функция для пресказаний, используя RandomForest
def predict_with_rf(models, X):
    preds = []
    for model in models:
        preds.append(model.predict(X))
    return np.mean(preds, axis = 0)

### Для того чтобы сделать предсказания надо в правильном виде задать X и выполнить ячейки с загузкой моделей и вызовом функции предсказания для требуемой переменной 

## K

In [ ]:
X, y = data[fts+[str(x) for x in range(300)]], data['absolute_permeability']

Так должны выглядеть данные, подаваемые на вход: сначала идут признаки (fts), затем профиль пористости (колонки 0-299)

In [ ]:
X

,.porosityConnected,.area_by_valume,.porosity_local.mean,.porosity_local.std,.porosity_local.mode,.porosity_micp.mean,.porosity_micp.std,.porosity_micp.mode,_qmicp0.25,_qmicp0.5,_qmicp0.75,_qlocal0.25,_qlocal0.5,_qlocal0.75,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,...,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
0,0.211851,30093.052469,22.131000,10.880222,20.2500,17.810768,9.657943,15.7500,12.479358,15.710703,19.741443,15.204028,20.384081,26.945441,0.002267,0.002264,0.002261,0.002264,0.002243,0.002216,0.002202,0.002189,0.002206,0.002258,0.002334,0.002409,0.002486,0.002539,0.002555,0.002574,0.002581,0.002575,0.002529,0.002486,0.002457,0.002430,0.002433,0.002418,0.002407,0.002398,0.002385,0.002381,0.002359,0.002326,0.002276,0.002225,0.002205,0.002183,0.002181,0.002162,...,0.002205,0.002226,0.002267,0.002323,0.002345,0.002347,0.002339,0.002338,0.002337,0.002326,0.002290,0.002250,0.002184,0.002179,0.002163,0.002167,0.002180,0.002220,0.002249,0.002278,0.002321,0.002354,0.002394,0.002415,0.002415,0.002393,0.002378,0.002377,0.002383,0.002376,0.002332,0.002290,0.002271,0.002274,0.002240,0.002201,0.002169,0.002134,0.002115,0.002111,0.002154,0.002190,0.002206,0.002180,0.002123,0.002060,0.001983,0.001907,0.001869,0.001840
1,0.210722,30332.324074,21.465415,9.101433,20.2500,16.478338,6.570775,15.7500,12.416005,15.375915,19.448717,15.126773,20.426078,26.871444,0.002247,0.002244,0.002251,0.002252,0.002248,0.002215,0.002191,0.002187,0.002171,0.002140,0.002098,0.002064,0.002018,0.001986,0.001962,0.001945,0.001965,0.001990,0.002031,0.002084,0.002128,0.002155,0.002150,0.002132,0.002098,0.002054,0.002011,0.001956,0.001913,0.001878,0.001851,0.001840,0.001847,0.001855,0.001886,0.001896,...,0.002241,0.002222,0.002216,0.002194,0.002181,0.002159,0.002153,0.002131,0.002101,0.002073,0.002070,0.002087,0.002105,0.002159,0.002196,0.002230,0.002271,0.002305,0.002302,0.002317,0.002312,0.002292,0.002283,0.002294,0.002298,0.002303,0.002301,0.002292,0.002271,0.002242,0.002206,0.002171,0.002145,0.002100,0.002063,0.002063,0.002067,0.002072,0.002080,0.002092,0.002109,0.002117,0.002115,0.002098,0.002107,0.002093,0.002088,0.002095,0.002088,0.002076
2,0.210641,29575.425926,22.055133,9.821042,20.2500,17.814278,7.627593,20.2500,13.042620,16.955674,20.244408,15.282709,20.724134,27.646900,0.001754,0.001773,0.001776,0.001776,0.001806,0.001833,0.001876,0.001911,0.001938,0.001971,0.002024,0.002086,0.002116,0.002137,0.002155,0.002195,0.002255,0.002284,0.002333,0.002392,0.002447,0.002444,0.002391,0.002343,0.002324,0.002305,0.002290,0.002263,0.002238,0.002210,0.002192,0.002173,0.002164,0.002168,0.002214,0.002271,...,0.001788,0.001786,0.001785,0.001796,0.001814,0.001841,0.001879,0.001952,0.002004,0.002050,0.002079,0.002097,0.002117,0.002132,0.002130,0.002124,0.002117,0.002087,0.002072,0.002045,0.002030,0.002013,0.002005,0.001985,0.001963,0.001938,0.001903,0.001872,0.001830,0.001811,0.001796,0.001787,0.001762,0.001735,0.001747,0.001770,0.001779,0.001792,0.001822,0.001842,0.001860,0.001863,0.001878,0.001888,0.001913,0.001935,0.001969,0.002016,0.002082,0.002153
3,0.217476,30369.688272,22.325847,10.569668,20.2500,16.782527,6.152925,15.7500,12.763110,16.213480,19.730502,15.527208,20.843162,27.680412,0.001857,0.001899,0.001930,0.001963,0.001996,0.002008,0.001998,0.001993,0.001976,0.001935,0.001906,0.001884,0.001851,0.001822,0.001793,0.001784,0.001797,0.001834,0.001872,0.001914,0.001947,0.001987,0.002025,0.002070,0.002106,0.002143,0.002171,0.002165,0.002168,0.002164,0.002185,0.002201,0.002236,0.002271,0.002307,0.002355,...,0.002279,0.002270,0.002276,0.002270,0.002281,0.002277,0.002295,0.002312,0.002331,0.002349,0.002372,0.002410,0.002421,0.002432,0.002437,0.002460,0.002462,0.002459,0.002445,0.002412,0.002363,0.002327,0.002278,0.002239,0.002189,0.002165,0.002156,0.002153,0.0021

Загрузка моделей:

In [ ]:
models_ap, scalers_ap = [], []  
for i in range(5):
    models_ap.append(load_model(f'./permeability_models/model_permeability{i}.h5'))
    scalers_ap.append([load(f'./permeability_models/scaler_1_permeability{i}.joblib'),
                       load(f'./permeability_models/scaler_2_permeability{i}.joblib')])     

In [ ]:
# предсказание
predictions = predict_with_nn(models_ap, scalers_ap, X) 

In [ ]:
mrpe(y,predictions)

NameError: name 'mrpe' is not defined

## Soi

In [ ]:
X, y = data[fts+[str(x) for x in range(300)]], data['Soi']

In [ ]:
#Загрузка моделей
models_soi, scalers_soi = [], []
for i in range(5):
    models_soi.append(load_model(f'./soi_models/model_soi{i}.h5'))
    scalers_soi.append([load(f'./soi_models/scaler_1_soi{i}.joblib'),
                       load(f'./soi_models/scaler_2_soi{i}.joblib')])    

In [ ]:
# предсказание
predictions = predict_with_nn(models_soi, scalers_soi, X) 

In [ ]:
mrpe(y,predictions)

0.1655441389702709

## FF

In [ ]:
X, y = data[fts+[str(x) for x in range(300)]], data['formation_factor'] 

In [ ]:
#Загрузка моделей
models_ff, scalers_ff = [], []
for i in range(5):
    models_ff.append(load_model(f'./formation_factor_models/model_ff{i}.h5'))
    scalers_ff.append([load(f'./formation_factor_models/scaler_1_ff{i}.joblib'),
                       load(f'./formation_factor_models/scaler_2_ff{i}.joblib')])    

In [ ]:
# предсказание
predictions = predict_with_nn(models_ff, scalers_ff, X) 

In [ ]:
mrpe(y,predictions)

0.10791295025708035

## a

In [ ]:
X, y = data[fts+[str(x) for x in range(300)]], data['a'] 

In [ ]:
#Загрузка моделей
models_a, scalers_a = [], []
for i in range(5):
    models_a.append(load_model(f'./a_models/model_a{i}.h5'))
    scalers_a.append([load(f'./a_models/scaler_1_a{i}.joblib'),
                       load(f'./a_models/scaler_2_a{i}.joblib')])    

In [ ]:
# предсказание
predictions = predict_with_nn(models_a, scalers_a, X) 

In [ ]:
mrpe(y,predictions)

0.1753201877116449

## b

In [ ]:
X, y = data[fts+[str(x) for x in range(300)]], data['b'] 

In [ ]:
#Загрузка моделей
models_b, scalers_b= [], []
for i in range(5):
    models_b.append(load_model(f'./b_models/model_b{i}.h5'))
    scalers_b.append([load(f'./b_models/scaler_1_b{i}.joblib'),
                       load(f'./b_models/scaler_2_b{i}.joblib')])  

In [ ]:
# предсказание
predictions = predict_with_nn(models_b, scalers_b, X) 

In [ ]:
mrpe(y,predictions)

0.25225371661081764

## krw_max

In [ ]:
X, y = data[fts+[str(x) for x in range(300)]], data['krw_max'] 

In [ ]:
#Загрузка моделей
models_krw, scalers_krw = [], []
for i in range(5):
    models_krw.append(load_model(f'./krw_models/model_krw_max{i}.h5'))
    scalers_krw.append([load(f'./krw_models/scaler_1_krw_max{i}.joblib'),
                       load(f'./krw_models/scaler_2_krw_max{i}.joblib')])  

In [ ]:
# предсказание
predictions = predict_with_nn(models_krw, scalers_krw, X) 

In [ ]:
mrpe(y,predictions)

0.26214678525456325

## sor

In [ ]:
X, y = data[fts+[str(x) for x in range(300)]], data['sor'] 

In [ ]:
models_sor = []
for i in range(5):
    models_sor.append(load(f'./sor_models/model_sor{i}.joblib'))

In [ ]:
predictions = predict_with_rf(models_sor, X)

In [ ]:
mrpe(y,predictions)

0.21307698477145406